In [8]:
import pandas as pd
import numpy as np
import scipy.stats as ss
import numpy as np

In [9]:
df = pd.read_csv('/Users/guapofish/Desktop/Factor Investing/winsor_factors_univariate.csv') 
    #556457*47
list_of_factors = ['b_m', 'e_p', 'cf_p', 'o', 'no', 'i_a', 'noa', 'pia', 'ig', 'nsi', 'ivg', 'ivc', 'oa', 'poa', 'pta', 'gpa',
           'oc_a', 'ol', 'R6', 'R11', 'sue', 'roe', 'roa', 'nei']
tc = False
if tc:
    # if self.tc == True 
    factors = df[['date', 'PERMNO', 'gvkey', 'equity', 'forward_r', 'fr'] + list_of_factors]
else:
    
    factors = df[['date', 'PERMNO', 'gvkey', 'equity', 'forward_r'] + list_of_factors]
        #556457*6

/Users/guapofish/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (29) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [11]:
factors = factors.replace([np.inf, -np.inf], 0)    #replace inf and -inf to 0  #556457*6
factors = factors.dropna(axis=0)                         #remove row contains missing value  #535140*6
factors = factors[factors.equity != 0]       #remove row of [equity] == 0  #535140*6
factors = factors.drop_duplicates(subset=['date', 'gvkey'])   #only consider ['date'] and ['gvkey'],  #535140*6
factors = factors.set_index('date')               #set date as index    535140*5
index = pd.to_datetime(factors.index, format = '%Y/%m/%d')     #change date format to '%Y/%M/%D'
factors = factors.set_index(index.date) 
dates = sorted(factors.index.unique())  #571      1971/6/30 - 2018/12/31

In [5]:
#x: z-score 
x = np.array([ss.zscore(factors[factors.index == i][list_of_factors].as_matrix(), axis=0) for i in dates]) 
#    dfx = pd.DataFrame(x)
#    dfx[[0,0]].shape  #572*3
#    x is nxk where N keeps changing
    
vv = [len(i) if not np.isnan(i).any() else 0 for i in x] 
index = [i for i,v in enumerate(vv) if v>100] 
    
dates = np.array(dates)[index]   #564

factors = factors[factors.index.isin(dates)] #534910*5
x = np.array([ss.zscore(factors[factors.index == i][list_of_factors].as_matrix(), axis=0) for i in dates])
r = np.array([np.array(factors[factors.index == i]['forward_r']) for i in dates])
    
gamma=3
T = len(dates) #246
K = len(list_of_factors)      #24
    # x and r are lists of length T
    # standardize each xt
    # x is nxk where N keeps changing

pivot_table = factors.pivot_table(values='equity', columns='gvkey', index=factors.index)
    # dataframe: 246*5305

    
weights = np.array(pivot_table) 
    
w_dash = np.nan_to_num(weights)

/Users/guapofish/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
/Users/guapofish/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  del sys.path[0]


In [9]:
theta_m = []
for i in range(2,T+1):
    w_sum = np.array(w_dash.sum(axis=1))
    summ = w_sum[:, np.newaxis] #246*1
    w_bar = weights / summ #246*5305
    
    r_bm = [np.dot(w_bar[j][~np.isnan(w_bar[j])], r[j]).T for j in range(i)]
    #print(r_bm)   
    xrt = [np.dot(x[j].T, r[j]) / x[j].shape[0] for j in range(i)]
    #print(xrt)
    mean_xrt = np.mean(xrt, axis=0)
    #print(mean_xrt)
    q1 = [xrt[j] - mean_xrt for j in range(i)]
    #print(q1)
    
    b = np.sum([np.dot(q1[j].reshape((K, 1)), q1[j].reshape((1, K))) for j in range(i)], axis=0) / (i+1)
    b_inv = np.linalg.inv(b)
    mean_rbm = np.mean(r_bm, axis=0)
    #print(mean_rbm)
    q2 = [r_bm[j] - mean_rbm for j in range(i)]
    b2 = np.sum([np.dot(q1[j], q2[j]) for j in range(i)], axis=0) / (i+1)
    c = np.dot(b_inv, b2)
    t1 = np.dot(b_inv, mean_xrt) / gamma
    theta = t1 - c
    theta_m.append(theta)


theta_m = pd.DataFrame(theta_m , columns = list_of_factors)
theta_m.set_index(dates[1:], inplace = True)

In [10]:
theta_m

,b_m,e_p,cf_p,o,no,i_a,noa,pia,ig,nsi,...,pta,gpa,oc_a,ol,R6,R11,sue,roe,roa,nei
1974-05-31,1.248039e+18,-6.638165e+18,-2.586153e+17,-3.203714e+18,-2.514840e+18,-1.604898e+18,1.777906e+18,-2.116680e+18,-4.299100e+17,4.189620e+18,...,-1.852695e+18,6.681712e+16,1.244706e+18,3.233210e+18,-8.814801e+17,-1.467290e+18,-2.968713e+18,-5.104812e+17,2.241071e+18,1.463121e+18
1974-06-28,-2.577224e+18,1.150942e+19,-1.340979e+19,-3.463793e+19,-3.436799e+18,1.283898e+19,1.840929e+19,-7.818744e+18,-1.165827e+19,6.854317e+18,...,-3.440355e+18,-1.913617e+18,-5.090399e+18,-1.325026e+19,6.322445e+18,-2.807288e+17,2.330089e+19,-1.060177e+19,-5.170020e+18,-4.370730e+18
1974-07-31,-6.782555e+16,1.390849e+17,-4.355588e+17,1.738496e+18,-3.517985e+16,-2.368584e+17,-2.956692e+17,1.875986e+17,4.713403e+17,5.120575e+17,...,1.554815e+17,-5.315266e+16,-1.329460e+17,-3.353635e+17,2.466559e+16,-3.123513e+17,1.284201e+17,9.122508e+16,-1.092908e+17,4.015556e+17
1974-08-30,-5.177218e+17,6.858997e+17,-3.097968e+16,3.811185e+17,6.551369e+17,5.771381e+17,3.290848e+16,-3.801886e+16,-2.662830e+17,1.205667e+17,...,-6.688375e+17,-2.787162e+17,-5.136933e+17,3.289800e+17,1.664046e+17,-1.971324e+16,4.872438e+17,-6.736057e+17,1.283684e+18,2.657952e+17
1974-09-30,1.629047e+17,-4.775098e+17,5.105185e+17,1.838647e+17,1.015823e+18,8.640304e+17,7.561062e+17,3.327038e+17,-3.980631e+18,-3.121065e+16,...,6.359992e+17,-3.247825e+17,1.796781e+18,-1.120871e+18,1.424536e+17,4.969186e+17,6.296844e+17,-4.456348e+17,2.149652e+17,-6.032942e+17
1974-10-31,1.884455e+16,-2.411869e+16,1.359036e+17,2.208798e+17,-2.256466e+17,2.151945e+17,3.338897e+17,1.524694e+17,-6.824239e+17,-5.889698e+17,...,1.897357e+17,2.635809e+13,3.987558e+17,-2.899522e+17,-1.309766e+17,6.940218e+16,2.318310e+17,2.262104e+17,-1.649421e+17,-2.454606e+17
1974-11-29,3.250482e+16,-1.103267e+17,-4.658487e+16,-1.546137e+17,-4.305857e+15,4.200270e+16,9.411113e+16,2.164829e+16,2.801297e+17,-2.453657e+16,...,-1.319687e+17,6.034981e+16,-8.626798e+16,-2.866175e+16,2.429709e+16,5.671662e+16,-1.106060e+17,5.742272e+16,-5.708858e+16,-2.041889e+17
1974-12-31,6.105750e+16,3.235505e+16,-1.592736e+17,1.819469e+17,-2.183337e+16,8.666599e+16,1.229688e+17,-3.562405e+17,-3.388011e+17,2.818742e+17,...,-2.492303e+16,-1.895851e+16,1.653468e+17,8.103809e+16,8.757182e+16,-8.370999e+16,-1.206488e+16,-4.055300e+16,-5.635103e+16,2.044538e+17
1975-01-31,-1.478630e+17,1.826578e+17,1.551105e+17,-4.384476e+16,8.373228e+16,1.331914e+17,-4.867948e+17,-8.482693e+16,-3.240867e+17,-2.414635e+17,...,2.911033e+17,-3.110351e+17,8.545454e+16,-4.091515e+17,3.687524e+17,-2.136651e+17,4.267476e+17,5.732997e+16,9.698814e+16,-1.819648e+17
1975-02-28,4.457649e+15,-6.923467e+15,-1.388385e+16,-5.755254e+15,5.471265e+15,-3.283452e+16,2.901461e+16,-4.249306e+16,4.616438e+16,5.572894e+16,...,2.169854e+15,-1.737567e+16,4.703176e+15,1.761281e+16,4.758423e+15,-2.125628e+16,1.220861e+16,-8.237957e+15,2.436481e+16,-1.671746e+16


In [2]:
import pandas as pd
import numpy as np
import scipy.stats as stats
from sklearn import linear_model as lm

In [19]:
def portfolio(theta_file, output_file, sepresent=False, sub=None, long_only=False):

    # left out nei

    factors = ['sue', 'roe', 'roa', 'nei', 'b_m', 'e_p', 'cf_p', 'o', 'no', 'i_a', 'noa', 'pia', 'ig', 'nsi', 'ivg',
               'ivc', 'oa', 'poa', 'pta', 'gpa', 'oc_a', 'ol', 'R6', 'R11']

    thetaresults = pd.read_csv(theta_file)
    thetaresults.index = thetaresults.factors

    rf = pd.read_csv('FF_Rf.csv')
    rf.columns.values[0] = 'date'
    rf = rf.copy()
    rf.index = rf.date
    rf['RF_f'] = rf.RF.shift(-1)
    rf = rf.RF_f
    rf = rf/100.0

    if sepresent == False:
        ind = ['theta', 'mean_r', 'sigma_r', 'min_r', '25%_r', '50%_r', '75%_r', 'max_r', 'w1', 'max_w', 'min_w',
           'negw_sum','negw_count', 'alpha', 'beta', 's', 't-stat']

    else:
        ind = ['theta', 'se', 'p-value', 'mean_r', 'sigma_r', 'min_r', '25%_r', '50%_r', '75%_r', 'max_r', 'w1',
               'max_w', 'min_w',
               'negw_sum', 'negw_count', 'alpha', 'beta', 's', 't-stat']

    df2 = pd.DataFrame()

    for i in factors:
        #print(i)
        if sub is not None:
            f = pd.read_csv(i+sub+'.csv')
        else:
            f = pd.read_csv(i + '.csv')
        #sue.forward_r = ((1+sue.forward_r)**4)-1
        f.columns.values[0] = 'date'
        f = f.copy()
        f['r_bar'] = f.w_bar * f.forward_r
        f['w'] = f.w_bar + f.w_active
        if not long_only:
            f['abs_w'] = f.w.abs()
        f['abs_wbar'] = f.w_bar.abs()


        if long_only:

            def lo(a):
                maxw = np.where(a['w'] > 0, a['w'], 0)
                a['w'] = maxw / np.sum(maxw)
                return a

            f = f.groupby('date').apply(lo)
            f['abs_w'] = f.w.abs()
            f['r_tot'] = f.w * f.forward_r

        a = f.groupby('date')

        def describ(g):
            df = pd.DataFrame()

            df['r_tot'] = g['r_tot'].sum()
            df['r_bar'] = g['r_bar'].sum()
            df['w1'] = g['abs_w'].mean()
            df['w1_bar'] = g.abs_wbar.mean()
            df['w_max'] = g.w.max()
            df['w_bar_max'] = g.w_bar.max()
            df['w_min'] = g.w.min()
            df['w_bar_min'] = g.w_bar.min()
            df['neg_w'] = pd.Series(a.w.apply(lambda x: (x[x < 0]).sum()))
            df['count_neg'] = a.w.apply(lambda x: (x < 0).mean())
            df['excess'] = df.r_tot - df.r_bar

            return df


        df = describ(a)
        rf_ = rf[rf.index>=df.index[0]]
        rf_ = rf_[rf_.index<=rf_.index[df.shape[0]-1]]
        rf_.index = df.index
        df = df.join(rf_)
        df = df.dropna()

        regr = lm.LinearRegression()
        regr.fit(np.array(df.r_bar-df.RF_f).reshape(-1,1), df.r_tot-df.RF_f)
        ypredict = regr.predict((np.array(df.r_bar - df.RF_f).reshape(-1, 1)))
        error = (df.r_tot-df.RF_f) - ypredict

        if sepresent:
            d = [thetaresults.theta[i], thetaresults.se[i], stats.norm.sf(np.abs(thetaresults.theta[i]/thetaresults.se[i]))*2,
                 (1 + df.r_tot.mean())**12 - 1, df.r_tot.std() * np.sqrt(12),
                 (1 + df.r_tot.min())**12 - 1] + [((1 + df.r_tot.quantile(x))**12 - 1) for x in np.array([.25,.5,.75])] + \
                [(1 + df.r_tot.max())**12 - 1, df.w1.mean() *100, df.w_max.mean() *100, df.w_min.mean() *100, df.neg_w.mean(),
                 df.count_neg.mean(), regr.intercept_, regr.coef_[0], np.std(error), regr.intercept_*np.sqrt(df.shape[0])/np.std(error)]

            dbar = [None, None, None, (1 + df.r_bar.mean())**12 - 1, df.r_bar.std() * np.sqrt(12), (1 + df.r_bar.min())**12 - 1] + \
                   [((1 + df.r_bar.quantile(x))**12 - 1) for x in np.array([.25,.5,.75])] + \
                [(1 + df.r_bar.max())*12 - 1, df.w1_bar.mean() * 100, df.w_bar_max.mean() * 100, df.w_bar_min.mean() * 100,
                 None, None, None, None, None, None]
        else:
            d = [thetaresults.theta[i],
                 (1 + df.r_tot.mean())**12 - 1, df.r_tot.std() * np.sqrt(12),
                 (1 + df.r_tot.min())**12 - 1] + [((1 + df.r_tot.quantile(x))**12 - 1) for x in np.array([.25,.5,.75])] + \
                [(1 + df.r_tot.max())**12 - 1, df.w1.mean() *100, df.w_max.mean() *100, df.w_min.mean() *100, df.neg_w.mean(),
                 df.count_neg.mean(), regr.intercept_, regr.coef_[0], np.std(error), regr.intercept_*np.sqrt(df.shape[0])/np.std(error)]

            dbar = [None, (1 + df.r_bar.mean())**12 - 1, df.r_bar.std() * np.sqrt(12), (1 + df.r_bar.min())**12 - 1] + \
                   [((1 + df.r_bar.quantile(x))**12 - 1) for x in np.array([.25,.5,.75])] + \
                [(1 + df.r_bar.max())*12 - 1, df.w1_bar.mean() * 100, df.w_bar_max.mean() * 100, df.w_bar_min.mean() * 100,
                 None, None, None, None, None, None]

        df2[i] = d
        if i=='b_m':
            df2['benchmark'+i] = dbar

    #df2 = pd.DataFrame({'factor':d, 'benchmark':dbar}, index = index)

    df2.index = ind

    df2.to_csv(output_file)

In [22]:
from __future__ import division
import numpy as np
import scipy.stats as ss
import pandas as pd
import scipy.sparse as _sparse
import multiprocessing as _multiprocessing
import logging

# it would take a lot of time to load the data again and again. so later find a way to customize list
# when making this change create a function to process the data

# when adding bootstrapping, think about not having to standardize again and again
class Score(object):
    # factors should have a column for equity and forward return

    def __init__(self, df, gamma, list_of_factors, tc=False):
        self.df = df

        self.gamma = gamma
        self.list = list_of_factors
        self.tc = tc

        # processing
        if self.tc:
            self.factors = self.df[['date', 'PERMNO', 'gvkey', 'equity', 'forward_r', 'fr'] + self.list]
        else:
            self.factors = self.df[['date', 'PERMNO', 'gvkey', 'equity', 'forward_r'] + self.list]
        self.factors = self.factors.replace([np.inf, -np.inf], 0)
        self.factors = self.factors.dropna()
        self.factors = self.factors[self.factors.equity != 0]
        self.factors = self.factors.drop_duplicates(subset=['date', 'gvkey'])
        self.factors = self.factors.set_index('date')
        index = pd.to_datetime(self.factors.index, format = '%Y/%m/%d')
        self.factors = self.factors.set_index(index.date)
        self.dates = sorted(self.factors.index.unique())

        self.x = np.array([ss.zscore(self.factors[self.factors.index == i][self.list].as_matrix(), axis=0) for i in self.dates])
        vv = [len(i) if not np.isnan(i).any() else 0 for i in self.x]
        index = [i for i,v in enumerate(vv) if v>100]
        self.dates = np.array(self.dates)[index]
        self.factors = self.factors[self.factors.index.isin(self.dates)]
        self.x = np.array([ss.zscore(self.factors[self.factors.index == i][self.list].as_matrix(), axis=0) for i in self.dates])
        self.r = np.array([np.array(self.factors[self.factors.index == i]['forward_r']) for i in self.dates])


# have a function for theta using a universal x and r
# the main theta value, save it intialization and print not return. instead should return a weights dataframe
# i dont want the weights dataframe when bootstrapping
# a single bootstrapping function. Alternatives to loops>? check bootstrapping packages source codes and maybe use multiprocessing

    def gettheta(self, csv_path = None):
        T = len(self.dates)
        K = len(self.list)
        # x and r are lists of length T
        # standardize each xt
        # x is nxk where N keeps changing
        weights = np.array(self.factors.pivot_table(values='equity', columns='gvkey', index=self.factors.index))
        w_dash = np.nan_to_num(weights)
        w_sum = np.array(w_dash.sum(axis=1))
        w_bar = weights / w_sum[:, np.newaxis]
        self.w_bar = w_bar
        r_bm = [np.dot(w_bar[i][~np.isnan(w_bar[i])], self.r[i]).T for i in range(T)]
        xrt = [np.dot(self.x[i].T, self.r[i]) / self.x[i].shape[0] for i in range(T)]
        mean_xrt = np.mean(xrt, axis=0)
        q1 = [xrt[i] - mean_xrt for i in range(T)]
        b = np.sum([np.dot(q1[i].reshape((K, 1)), q1[i].reshape((1, K))) for i in range(T)], axis=0) / T
        b_inv = np.linalg.inv(b)
        mean_rbm = np.mean(r_bm, axis=0)
        q2 = [r_bm[i] - mean_rbm for i in range(T)]
        b2 = np.sum([np.dot(q1[i], q2[i]) for i in range(T)], axis=0) / T
        c = np.dot(b_inv, b2)
        t1 = np.dot(b_inv, mean_xrt) / self.gamma
        theta = t1 - c
        self.theta = theta

        # output weights for further analysis
        # think about inheritance to analysis class
        permno = [self.factors[self.factors.index == i]['PERMNO'] for i in self.dates]
        gvkey = [self.factors[self.factors.index == i]['gvkey'] for i in self.dates]
        equity = [self.factors[self.factors.index == i]['equity'] for i in self.dates]
        if self.tc:
            orig_r = [self.factors[self.factors.index == i]['tc'] for i in self.dates]
        w_active = [np.dot(np.reshape(self.x[i], (self.x[i].shape[0],K)), theta) / self.x[i].shape[0] for i in range(T)]
        w_tot = [w_bar[i][~np.isnan(w_bar[i])] + w_active[i] for i in range(T)]
        r_tot = [np.multiply(w_tot[i], self.r[i]) for i in range(T)]
        r_s = [r_tot[i].sum() for i in range(T)]
        if self.tc:
            l = pd.DataFrame({'permno': pd.concat((i for i in permno)), 'gvkey': pd.concat((i for i in gvkey)),
                              'forward_r': np.hstack((i for i in self.r)),
                              'tc': pd.concat((i for i in orig_r)),
                              'w_bar': np.hstack((i for i in w_bar[~np.isnan(w_bar)])),
                              'w_active': np.hstack((i for i in w_active)), 'r_tot': np.hstack((i for i in r_tot)),
                              'w_tot': np.hstack((i for i in w_tot)), 'equity': pd.concat((i for i in equity))})
        else:
            l = pd.DataFrame({'permno': pd.concat((i for i in permno)), 'gvkey': pd.concat((i for i in gvkey)), 'forward_r': np.hstack((i for i in self.r)),
                          'w_bar': np.hstack((i for i in w_bar[~np.isnan(w_bar)])),
                          'w_active': np.hstack((i for i in w_active)), 'r_tot': np.hstack((i for i in r_tot)),
                          'w_tot': np.hstack((i for i in w_tot)), 'equity': pd.concat((i for i in equity))})

        print('Theta: {0}\nMean Monthly Return: {1}\n STD Monthly Return: {2}'.format(theta, np.mean(r_s), np.std(r_s)))

        if csv_path is not None:
            l.to_csv(csv_path)

        return l

    def resample(self):
        # number of dates
        date_num = np.size(self.dates)
        # pick resample dates
        sample_index = np.random.choice(date_num, date_num, replace=True)
        x = self.x[sample_index]
        r = self.r[sample_index]
        w_bar = self.w_bar[sample_index]
        return x, r, w_bar

    def _theta(self, x, r, w_bar):
        T = len(self.dates)
        K = len(self.list)
        r_bm = [np.dot(w_bar[i][~np.isnan(w_bar[i])], r[i]).T for i in range(T)]
        xrt = [np.dot(x[i].T, r[i]) / x[i].shape[0] for i in range(T)]
        mean_xrt = np.mean(xrt, axis=0)
        q1 = [xrt[i] - mean_xrt for i in range(T)]
        b = np.sum([np.dot(q1[i].reshape((K, 1)), q1[i].reshape((1, K))) for i in range(T)], axis=0) / T
        b_inv = np.linalg.inv(b)
        mean_rbm = np.mean(r_bm, axis=0)
        q2 = [r_bm[i] - mean_rbm for i in range(T)]
        b2 = np.sum([np.dot(q1[i], q2[i]) for i in range(T)], axis=0) / T
        c = np.dot(b_inv, b2)
        t1 = np.dot(b_inv, mean_xrt) / self.gamma
        theta = t1 - c
        return theta

    def bootstrapping(self, size=1000):
        results = []
        for i in range(size):
            a, r, w_bar = self.resample()
            results.append(self._theta(a,r,w_bar))
        return ss.sem(np.array(results, dtype=np.float), nan_policy='omit')



In [23]:
import pandas as pd
import numpy as np

factors = ['b_m', 'e_p', 'cf_p', 'o', 'no', 'i_a', 'noa', 'pia', 'ig', 'nsi', 'ivg', 'ivc', 'oa', 'poa', 'pta', 'gpa',
           'oc_a', 'ol', 'R6', 'R11', 'sue', 'roe', 'roa', 'nei']
'''
Aversion = 3; optimal
'''
def scores(file, factors, gamma=3):

    df = pd.read_csv(file)
h
    theta = []
    se = []

    for i in factors:
        print(i)
        f = Score(df, gamma, [i])
        t = f.gettheta(i+'.csv')
        theta.append(f.theta[0])
        se.append(None)
        #se.append(f.bootstrapping(size=1000)[0])
    dfse = pd.DataFrame({'factors':factors, 'theta':theta, 'se':se})
    dfse.to_csv('univariate_theta.csv')

    return 'done'

scores('/Users/guapofish/Desktop/Factor Investing/winsor_factors_univariate.csv', factors)





b_m


/Users/guapofish/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:37: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
/Users/guapofish/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:42: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


Theta: [4.33814517]
Mean Monthly Return: 0.02309741734976739
 STD Monthly Return: 0.079422420177775
e_p
Theta: [5.01414503]
Mean Monthly Return: 0.02200831227021417
 STD Monthly Return: 0.07694281658247289
cf_p
Theta: [4.56982577]
Mean Monthly Return: 0.020612399810014264
 STD Monthly Return: 0.07300840467317404
o
Theta: [4.09880344]
Mean Monthly Return: 0.01371315274620871
 STD Monthly Return: 0.05024666339784051
no
Theta: [4.44137232]
Mean Monthly Return: 0.015193380838167186
 STD Monthly Return: 0.054262394929982104
i_a
Theta: [-7.28748625]
Mean Monthly Return: 0.025913046316896494
 STD Monthly Return: 0.07694042150064893
noa
Theta: [-8.73165231]
Mean Monthly Return: 0.034284143768014634
 STD Monthly Return: 0.10193771958582823
pia
Theta: [-14.48043171]
Mean Monthly Return: 0.051672879258505236
 STD Monthly Return: 0.11855182748223618
ig
Theta: [-6.43822639]
Mean Monthly Return: 0.017408303650311405
 STD Monthly Return: 0.06200601064299096
nsi
Theta: [-7.37065874]
Mean Monthly Retur

/Users/guapofish/anaconda3/lib/python3.7/site-packages/scipy/stats/stats.py:2253: RuntimeWarning: invalid value encountered in true_divide
  return (a - mns) / sstd


Theta: [2.36417744]
Mean Monthly Return: 0.015323521604680554
 STD Monthly Return: 0.06278459134154694
R11
Theta: [2.08003114]
Mean Monthly Return: 0.014551063784557334
 STD Monthly Return: 0.062242339461185285
sue
Theta: [15.46707479]
Mean Monthly Return: 0.04413591539613724
 STD Monthly Return: 0.11519592550936539
roe
Theta: [3.28727751]
Mean Monthly Return: 0.012629948400512549
 STD Monthly Return: 0.0546583128596517
roa
Theta: [1.82869298]
Mean Monthly Return: 0.011356420807429895
 STD Monthly Return: 0.05040747178295947
nei
Theta: [4.63545346]
Mean Monthly Return: 0.014493232610824977
 STD Monthly Return: 0.06157203743432405


'done'